In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import ( Input, Embedding, Flatten, Dot, Add, Concatenate, 
        Dense, Dropout, Activation, Multiply )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.initializers import RandomUniform

In [2]:
ratings = pd.read_csv('ml-100k/ratings.csv', sep=',',encoding='latin-1')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

m_unique = ratings.movieId.unique()
movie2Id = { movieId : index for index, movieId in enumerate(m_unique)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2Id[x])

In [4]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(610, 9724)

In [5]:
n_factors = 50

In [6]:
np.random.seed = 42

In [7]:
msk = np.random.rand(len(ratings)) < 0.8

In [8]:
trn = ratings[msk]
val = ratings[~msk]

# Embeddings Model

In [9]:
user_in = Input(shape=(1,), dtype='int64', name='user_input')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_input')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(movie_in)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

Instructions for updating:
Use tf.cast instead.


In [11]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80514 samples, validate on 20322 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
80514/80514 [==============================] - 21s 257us/sample - loss: 9.7480 - val_loss: 3.6881
Epoch 2/3
80514/80514 [==============================] - 21s 263us/sample - loss: 2.2816 - val_loss: 1.9082
Epoch 3/3
80514/80514 [==============================] - 16s 200us/sample - loss: 1.3391 - val_loss: 1.5796


# Bias with Embeddings Model

In [12]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [13]:
user_in, u = embedding_input('user_input', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 1e-5)

In [14]:
def create_bias(inp, n_in):
    return Flatten()(Embedding(n_in, 1, input_length=1)(inp))

In [15]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [16]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
x = Add()([x, ub])
x = Add()([x, mb])
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [17]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80514 samples, validate on 20322 samples
Epoch 1/3
80514/80514 [==============================] - 18s 221us/sample - loss: 8.6842 - val_loss: 2.9968
Epoch 2/3
80514/80514 [==============================] - 17s 206us/sample - loss: 1.8561 - val_loss: 1.5395
Epoch 3/3
80514/80514 [==============================] - 17s 206us/sample - loss: 1.1179 - val_loss: 1.2781


# Neural Net

In [18]:
user_in, u = embedding_input('user_input', n_users , n_factors, 1e-5)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 1e-5)

In [19]:
x = Concatenate()([u, m])
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)
x = Dropout(rate=0.75)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=4,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80514 samples, validate on 20322 samples
Epoch 1/4
80514/80514 [==============================] - 16s 203us/sample - loss: 2.0268 - val_loss: 0.8454
Epoch 2/4
80514/80514 [==============================] - 17s 211us/sample - loss: 1.2096 - val_loss: 0.8168
Epoch 3/4
80514/80514 [==============================] - 16s 204us/sample - loss: 1.0467 - val_loss: 0.8159
Epoch 4/4
80514/80514 [==============================] - 17s 207us/sample - loss: 0.9004 - val_loss: 0.7945
